In [1]:
import sqlite3
import pandas as pd

In [2]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [3]:
conn = sqlite3.connect('parch-and-posey.db')

In [4]:
cursor = conn.cursor()
cursor.execute('''
select * from sqlite_master where type = "table";
''')
columns = [col[0] for col in cursor.description]
data = cursor.fetchall()
cursor.close()

In [5]:
pd.DataFrame(data, columns=columns)

,type,name,tbl_name,rootpage,sql
0,table,web_events,web_events,2,"CREATE TABLE web_events (\tid integer,\taccoun..."
1,table,sales_reps,sales_reps,92,"CREATE TABLE sales_reps (\tid integer,\tname b..."
2,table,region,region,93,"CREATE TABLE region (\tid integer,\tname bpchar)"
3,table,orders,orders,94,"CREATE TABLE orders (\tid integer,\taccount_id..."
4,table,accounts,accounts,221,"CREATE TABLE accounts (\tid integer,\tname bpc..."


Task 1:

# Comparing a Row to Previous Row

In the previous video, Derek outlines how to compare a row to a previous or subsequent row. This technique can be useful when analyzing time-based events. Imagine you're an analyst at Parch & Posey and you want to determine how the current order's total revenue ("total" meaning from sales of all types of paper) compares to the next order's total revenue.

Modify Derek's query from the previous video in the SQL Explorer below to perform this analysis. You'll need to use occurred_at and total_amt_usd in the orders table along with LEAD to do so. In your query results, there should be four columns: occurred_at, total_amt_usd, lead, and lead_difference.

In [8]:
# original

pd.read_sql_query(sql='''
SELECT account_id,
       standard_sum,
       LAG(standard_sum) OVER (ORDER BY standard_sum) AS lag,
       LEAD(standard_sum) OVER (ORDER BY standard_sum) AS lead,
       standard_sum - LAG(standard_sum) OVER (ORDER BY standard_sum) AS lag_difference,
       LEAD(standard_sum) OVER (ORDER BY standard_sum) - standard_sum AS lead_difference
FROM (
SELECT account_id,
       SUM(standard_qty) AS standard_sum
  FROM orders 
 GROUP BY 1
 ) sub;
''', con=conn)

,account_id,standard_sum,lag,lead,lag_difference,lead_difference
0,1901,0,NaN,79.0,NaN,79.0
1,3371,79,0.0,102.0,79.0,23.0
2,1961,102,79.0,116.0,23.0,14.0
3,3401,116,102.0,117.0,14.0,1.0
4,3741,117,116.0,123.0,1.0,6.0
...,...,...,...,...,...,...
345,1281,21943,21448.0,22099.0,495.0,156.0
346,1181,22099,21943.0,22936.0,156.0,837.0
347,3491,22936,22099.0,23321.0,837.0,385.0
348,2631,23321,22936.0,41617.0,385.0,18296.0


In [14]:
# my solution

pd.read_sql_query(sql='''
SELECT occurred_at,
       total_amt_usd_sum,
       LAG(total_amt_usd_sum) OVER (ORDER BY occurred_at) AS lag,
       LEAD(total_amt_usd_sum) OVER (ORDER BY occurred_at) AS lead,
       total_amt_usd_sum - LAG(total_amt_usd_sum) OVER (ORDER BY occurred_at) AS lag_difference,
       LEAD(total_amt_usd_sum) OVER (ORDER BY occurred_at) - total_amt_usd_sum AS lead_difference
FROM (
SELECT occurred_at,
       SUM(total_amt_usd) AS total_amt_usd_sum
  FROM orders 
 GROUP BY 1
 ) sub;
''', con=conn)

,occurred_at,total_amt_usd_sum,lag,lead,lag_difference,lead_difference
0,2013-12-04 04:22:44,627.48,NaN,2646.77,NaN,2019.29
1,2013-12-04 04:45:54,2646.77,627.48,2709.62,2019.29,62.85
2,2013-12-04 04:53:25,2709.62,2646.77,277.13,62.85,-2432.49
3,2013-12-05 20:29:16,277.13,2709.62,3001.85,-2432.49,2724.72
4,2013-12-05 20:33:56,3001.85,277.13,2802.90,2724.72,-198.95
...,...,...,...,...,...,...
6903,2017-01-01 21:04:25,892.85,2582.40,2666.79,-1689.55,1773.94
6904,2017-01-01 22:17:26,2666.79,892.85,1854.57,1773.94,-812.22
6905,2017-01-01 22:29:50,1854.57,2666.79,1932.85,-812.22,78.28
6906,2017-01-01 23:50:16,1932.85,1854.57,6451.76,78.28,4518.91


Solution:

SELECT occurred_at,
       total_amt_usd,
       LEAD(total_amt_usd) OVER (ORDER BY occurred_at) AS lead,
       LEAD(total_amt_usd) OVER (ORDER BY occurred_at) - total_amt_usd AS lead_difference
FROM (
SELECT occurred_at,
       SUM(total_amt_usd) AS total_amt_usd
  FROM orders 
 GROUP BY 1
) sub